In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
# pandas
import pandas as pd
import time

In [16]:
# Chrome Browser와 Chrome Driver Version 확인
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.get('https://www.starbucks.co.kr/store/store_map.do')

initbtn = driver.find_element_by_xpath('//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a')
initbtn.click()


time.sleep(1)

btns = driver.find_elements_by_class_name('set_sido_cd_btn')


df = pd.DataFrame(columns=['name', 'address'])

for btn in btns:
    btn.click()
    time.sleep(1)
    innerDf = pd.DataFrame(columns=['name', 'address'])
    try:
        innerBtn = driver.find_element_by_xpath('//*[@id="mCSB_2_container"]/ul/li[1]/a')
        innerBtn.click()
        time.sleep(4)
        html = driver.page_source
        # parse html
        soup = BeautifulSoup(html, 'html.parser')

        namelist = soup.select('.quickSearchResultBoxSidoGugun > li > strong')
        addrlist = soup.select('.quickSearchResultBoxSidoGugun > li > .result_details')

        # 이름, 주소, 전화번호 데이터프레임
        for i, name in enumerate(namelist):
            addr = addrlist[i].text.split('<br>')[0].strip()
            innerDf.loc[i] = [name.text.strip(), addr]
        
        outBtn = driver.find_element_by_xpath('//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a')
        outBtn.click()

        df = df.append(innerDf)

    except:
        break

df.to_csv('starbucks2.csv', encoding='utf-8', mode='w', index=False) 
    


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/tj/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
